In [45]:
#import libraries
import scipy.io as sio
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
%matplotlib inline
from pprint import pprint

#preliminary model libraries
from keras import regularizers
from keras.preprocessing import image
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.models import Model
from keras.applications.xception import (
    Xception, preprocess_input, decode_predictions)


#comparator libraries
from skimage import io, measure
from skimage.measure import compare_ssim
import cv2

In [93]:
#Define functions
path = os.path.join("./Car Data/cam_images2")

def newest(path):
    files = os.listdir(path)
    #print(files)
    paths = [os.path.join(path, basename) for basename in files]
    recentPath=max(paths, key=os.path.getctime)
    print(recentPath)
    return recentPath
newest(path)

./Car Data/cam_images2\cam_img (100) - Copy.jpg


'./Car Data/cam_images2\\cam_img (100) - Copy.jpg'

In [47]:
#Load Pre-trained model
model = Xception(
    include_top=True,
    weights='imagenet')
image_size = (299, 299, 3)

In [48]:
#load pre-trained model
VID = pd.read_csv("vehicle_categories.txt", sep='\t')
CID = pd.read_csv("clothing_categories.txt", sep='\t')

In [49]:
#Primary Model
data2 = []
ismatch = 0

image_size = (299, 299, 3)

image_path = newest(path)
img = image.load_img(image_path, target_size=image_size)

x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

predictions = model.predict(x)

result = decode_predictions(predictions, top=3)
topresult = result[0][0]
result2 = result[0][1]
result3 = result[0][2]

if (topresult[0] in VID['Serial'].values) or (result2[0] in VID['Serial'].values) or (result3[0] in VID['Serial'].values):
    imgclass = "Vehicle"
    if (topresult[0] in VID['Serial'].values):
        classp = str((round(topresult[2]*1000))/10)
    elif (result2[0] in VID['Serial'].values):
        classp = str((round(result2[2]*1000))/10)
    else: # (result3[0] in VID['Serial'].values):
        classp = str((round(result3[2]*1000))/10)
    ismatch = 1 
elif (topresult[0] in CID['Serial'].values) or (result2[0] in CID['Serial'].values) or (result3[0] in CID['Serial'].values):
    imgclass = "Person"
    if (topresult[0] in CID['Serial'].values):
        classp = str((round(topresult[2]*1000))/10)
    elif (result2[0] in CID['Serial'].values):
        classp = str((round(result2[2]*1000))/10)
    else: #(result3[0] in CID['Serial'].values):
        classp = str((round(result3[2]*1000))/10) 
    ismatch = 2
else:
    imgclass = "Other"# + str((round(topresult[2]*1000))/10)
    classp = str((round(topresult[2]*1000))/10)

data2.append([topresult[0],topresult[1],(round(topresult[2]*1000))/10,imgclass, classp, ismatch])
data2 = pd.DataFrame(data2, columns = ['Serial','Pred','Prob','Class','Class Pr','Match'])

./Car Data/cam_images2\cam_img (100) - Copy.jpg


In [50]:
Sarray = [0]
for scen in range(1,19):
    Sarray.append(io.imread('./Car Data/cam_base/S%s.jpg' %(scen)))
    
print(Sarray)

[0, array([[[142, 133,  74],
        [125, 116,  57],
        [141, 132,  73],
        ...,
        [ 87,  74,  40],
        [104,  91,  57],
        [112, 101,  56]],

       [[138, 129,  70],
        [130, 121,  62],
        [145, 139,  79],
        ...,
        [ 73,  60,  26],
        [124, 111,  77],
        [105,  94,  49]],

       [[129, 123,  63],
        [124, 118,  58],
        [135, 129,  69],
        ...,
        [109,  96,  61],
        [113, 100,  66],
        [ 94,  83,  38]],

       ...,

       [[ 50,  58,   0],
        [ 76,  84,  24],
        [120, 126,  66],
        ...,
        [103, 104, 108],
        [103, 104, 108],
        [103, 104, 108]],

       [[ 96, 104,  44],
        [ 74,  82,  22],
        [ 51,  56,   0],
        ...,
        [103, 104, 108],
        [103, 104, 108],
        [103, 104, 108]],

       [[ 56,  66,   5],
        [ 91,  99,  40],
        [ 93, 101,  42],
        ...,
        [103, 104, 108],
        [103, 104, 108],
        [103, 104, 1

In [51]:
print(image_path)

./Car Data/cam_images2\cam_img (100) - Copy.jpg


In [56]:

comparray = []
i1 =0

#cycle through photo backlog
for img1 in data2['Serial']:
#for img1 in range(120,130):
    i1+=1
    print(i1,data2[data2.Serial==img1].Match.item())
    
    #cycle through scenarios for each photo
    resultarray = []
    
    imgpathio = io.imread(image_path)
   
    if data2[data2.Serial==img1].Match.item() ==2:
        #if photo matched to person, run against S1 & S10
        for R in (1,10):
            scenario = Sarray[R]
            combossim = round(compare_ssim(imgpathio,scenario,multichannel=True)*1000)/10

            resultarray.append([R,combomse,combossim,"P"])
    else:   
        #if photo matched to vehicle, run against all
        for S in range(1,19):
            scenario = Sarray[S]
            combossim = round(compare_ssim(imgpathio,scenario,multichannel=True)*1000)/10

            resultarray.append([S,combossim,"V"])
    resultdf = pd.DataFrame(resultarray, columns = ['Scenario','SSIM','Type']) 

    #find best match
    Y = int(str(resultdf.loc[resultdf['SSIM'].idxmax()][0]).split('.')[0])
    Bestmatch = Y
    Bestssim = resultdf.loc[resultdf['SSIM'].idxmax()][1]
    Type = resultdf.loc[resultdf['SSIM'].idxmax()][2]
    #Mark high & Low
    
    #Person thresholds are different from vehicle thresholds 
    if data2[data2.Serial==img1].Match.item() ==2:#Is it a person
        if Bestssim >94:
            initialflag = 1
        elif Bestssim < 41:
            initialflag = 2
        else:
            initialflag = 0
    else:  #or is it a vehicle
        if Bestssim >82:
            initialflag = 1
        elif Bestssim < 44:
            initialflag = 2
        else:
            initialflag = 0
        
        
    comparray.append([i1,Bestmatch,Bestssim,initialflag,Type])

compdf = pd.DataFrame(comparray, columns = ['Photo', 'Best Match','SSIM','Flag','Type'])        
print(compdf) 

1 1
   Photo  Best Match  SSIM  Flag Type
0      1           1  56.0     0    V


In [63]:
newline = compdf.loc[0,:].tolist()

In [68]:
import csv
output = [newline]
print(output)
#with open("output.csv",'wb') as resultFile:
#    wr = csv.writer(resultFile, dialect='excel')
#    wr.writerows(output)

[[1, 1, 56.0, 0, 'V']]


In [91]:
with open("output.csv", "a") as outfile:
    outfile.write("\n")
    for entries in newline:
        outfile.write(str(entries) + ", ")
        

In [104]:
files = os.listdir(path)
paths = [os.path.join(path, basename) for basename in files]
recentPath=max(paths, key=os.path.getctime)
filetime = os.path.getctime(recentPath)

In [105]:
from datetime import datetime
ts = int(filetime)

# if you encounter a "year is out of range" error the timestamp
# may be in milliseconds, try `ts /= 1000` in that case
print(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

2019-03-28 22:49:39


In [124]:
import random
inputfile = newest(path)
newline = [False,"V/P",random.randint(1,101),random.randint(1,101)]
if newline[0]:
    print("Alert!")
with open("output.csv", "a") as outfile:
        outfile.write("\n")
        outfile.write(datetime.utcfromtimestamp(int(os.path.getctime(inputfile))).strftime('%Y-%m-%d %H:%M:%S'))
        for entries in newline:
            outfile.write(", ")
            outfile.write(str(entries))

./Car Data/cam_images2\cam_img (100) - Copy.jpg
